In [6]:
pip install nltk rouge-score scikit-learn sentence-transformers sacrebleu


Looking in indexes: https://nexus.iisys.de/repository/ki-awz-pypi-group/simple, https://pypi.org/simple
  Using cached rouge_score-0.1.2-py3-none-any.whl
  Obtaining dependency information for sentence-transformers from https://files.pythonhosted.org/packages/8b/c8/990e22a465e4771338da434d799578865d6d7ef1fdb50bd844b7ecdcfa19/sentence_transformers-3.3.1-py3-none-any.whl.metadata
  Using cached sentence_transformers-3.3.1-py3-none-any.whl.metadata (10 kB)
  Obtaining dependency information for sacrebleu from https://files.pythonhosted.org/packages/15/d8/e51d35bc863caa19ddeae48dfb890581a19326973ad1c9fa5dcfc63310f7/sacrebleu-2.4.3-py3-none-any.whl.metadata
  Using cached sacrebleu-2.4.3-py3-none-any.whl.metadata (51 kB)
  Obtaining dependency information for portalocker from https://files.pythonhosted.org/packages/3d/4c/4cb6bb4061910ac74c444be76e7d17dba97d9057030cca2f96947c3f7a0f/portalocker-3.0.0-py3-none-any.whl.metadata
  Using cached portalocker-3.0.0-py3-none-any.whl.metadata (8.5 kB)

In [5]:
pip install numpy pycocotools nltk


Looking in indexes: https://nexus.iisys.de/repository/ki-awz-pypi-group/simple, https://pypi.org/simple
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install git+https://github.com/salaniz/pycocoevalcap.git


Looking in indexes: https://nexus.iisys.de/repository/ki-awz-pypi-group/simple, https://pypi.org/simple
  Cloning https://github.com/salaniz/pycocoevalcap.git to /tmp/pip-req-build-pumdroyw
  Running command git clone --filter=blob:none --quiet https://github.com/salaniz/pycocoevalcap.git /tmp/pip-req-build-pumdroyw
  Resolved https://github.com/salaniz/pycocoevalcap.git to commit a24f74c408c918f1f4ec34e9514bc8a76ce41ffd
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for pycocotools>=2.0.2 from https://files.pythonhosted.org/packages/6a/03/6c0bf810a5df7876caaf11f5b113e7ffd4b2fa9767d360489c6fdcefe8e5/pycocotools-2.0.8-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached pycocotools-2.0.8-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.1 kB)
Using cached pycocotools-2.0.8-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (427 kB)
  Created wheel for pycocoevalcap: filename=pycocoevalcap-1.2-py3-non

In [3]:
import nltk

# Ensure required NLTK resources are downloaded
nltk.download('wordnet')
nltk.download('omw-1.4')  # Optional: for extended WordNet resources


[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jovyan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [16]:
!pip install bert-score


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://nexus.iisys.de/repository/ki-awz-pypi-group/simple, https://pypi.org/simple
  Obtaining dependency information for bert-score from https://files.pythonhosted.org/packages/c6/8c/bc5457de4c004b1a623b31f7bc8d0375fb699b7d67df11879098b4b7b7c8/bert_score-0.3.13-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.6 MB/s eta 0:00:00


In [4]:
import json
from sklearn.metrics import precision_score, recall_score, f1_score
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from sentence_transformers import SentenceTransformer, util
from nltk.translate.meteor_score import meteor_score
from pycocoevalcap.cider.cider import Cider
from math import exp
import numpy as np
from networkx import Graph
from networkx.algorithms.similarity import graph_edit_distance

# Load the dataset
def load_dataset(file_path):
    with open(file_path, "r") as file:
        return json.load(file)

# Preprocess text
def preprocess_text(text):
    if isinstance(text, list):
        return " ".join([str(t).strip() for t in text if t is not None])
    return str(text).strip() if text is not None else ""

# Exact Match
def exact_match_score(pred, ground_truth):
    pred = preprocess_text(pred).lower()
    ground_truth = preprocess_text(ground_truth).lower()
    return int(pred == ground_truth)

# BLEU Score with Smoothing
def bleu_metric(pred, ground_truth):
    pred = preprocess_text(pred)
    ground_truth = preprocess_text(ground_truth)
    smoothing_fn = SmoothingFunction().method1
    return sentence_bleu([ground_truth.lower().split()], pred.lower().split(), smoothing_function=smoothing_fn)

# ROUGE-L Score
def rouge_l_metric(pred, ground_truth):
    pred = preprocess_text(pred)
    ground_truth = preprocess_text(ground_truth)
    scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)
    scores = scorer.score(ground_truth.lower(), pred.lower())
    return scores['rougeL'].fmeasure

# Semantic Similarity
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

def semantic_similarity(pred, ground_truth):
    pred = preprocess_text(pred)
    ground_truth = preprocess_text(ground_truth)
    embeddings = model.encode([ground_truth, pred])
    return util.cos_sim(embeddings[0], embeddings[1]).item()

# METEOR Score
# METEOR Score
def meteor_metric(pred, ground_truth):
    pred = preprocess_text(pred).split()  # Tokenize into list of words
    ground_truth = preprocess_text(ground_truth).split()  # Tokenize into list of words
    return meteor_score([ground_truth], pred)


# CIDEr Score (using pycocoevalcap)
cider_scorer = Cider()

def cider_metric(pred, ground_truth):
    pred = preprocess_text(pred)
    ground_truth = preprocess_text(ground_truth)
    return cider_scorer.compute_score({"0": [ground_truth]}, {"0": [pred]})[0]

# Perplexity (Language Model Smoothness)
def perplexity(pred):
    tokens = preprocess_text(pred).split()
    n = len(tokens)
    if n == 0:
        return float('inf')
    word_probs = [1 / len(tokens)] * len(tokens)  # Simplified word probabilities
    log_prob = sum(np.log(word_probs))
    return exp(-log_prob / n)

# Coh-Metrix Proxy (Token Diversity)
def coh_metrix(pred):
    tokens = preprocess_text(pred).split()
    return len(set(tokens)) / len(tokens) if tokens else 0

# Graph Similarity for Structural Consistency
def build_graph(answer):
    graph = Graph()
    for entity in answer.split(","):
        parts = entity.strip().split(":")
        if len(parts) == 2:
            graph.add_edge(parts[0].strip(), parts[1].strip())
    return graph

def graph_similarity(pred, ground_truth):
    pred_graph = build_graph(preprocess_text(pred))
    gt_graph = build_graph(preprocess_text(ground_truth))
    try:
        similarity = 1 - (graph_edit_distance(pred_graph, gt_graph) / max(len(pred_graph), len(gt_graph)))
        return max(0.0, similarity)  # Ensure non-negative values
    except Exception:
        return 0.0

# Evaluation loop
def evaluate_dataset(dataset):
    metrics = {"qwen2vl": [], "llava": [], "kosmos": []}

    for entry in dataset:
        if isinstance(entry, dict):  # Check if entry is valid
            questions = entry.get("questions", [])
            for question_entry in questions:
                ground_truth = question_entry.get("ground_truth_answer", "")
                for model_key in ["generated_answer_qwen2vl", "generated_answer_llava", "generated_answer_kosmos"]:
                    model_name = model_key.split("_")[-1]
                    generated_answer = question_entry.get(model_key, "")

                    # Compute metrics
                    bleu = bleu_metric(generated_answer, ground_truth)
                    rouge_l = rouge_l_metric(generated_answer, ground_truth)
                    sim = semantic_similarity(generated_answer, ground_truth)
                    meteor = meteor_metric(generated_answer, ground_truth)
                    cider = cider_metric(generated_answer, ground_truth)
                    perp = perplexity(generated_answer)
                    coh = coh_metrix(generated_answer)
                    graph_sim = graph_similarity(generated_answer, ground_truth)

                    # Store results
                    metrics[model_name].append({
                        "question": question_entry.get("question", ""),
                        "ground_truth": ground_truth,
                        "generated_answer": generated_answer,
                        "bleu": bleu,
                        "rouge_l": rouge_l,
                        "semantic_similarity": sim,
                        "meteor": meteor,
                        "perplexity": perp,
                        "coh_metrix": coh,
                        "graph_similarity": graph_sim,
                    })

    return metrics

# Aggregated Metrics
def aggregate_metrics(metrics):
    aggregated = {}
    for model, results in metrics.items():
        bleu_scores = [r["bleu"] for r in results]
        rouge_l_scores = [r["rouge_l"] for r in results]
        semantic_similarities = [r["semantic_similarity"] for r in results]
        meteor_scores = [r["meteor"] for r in results]
        perplexities = [r["perplexity"] for r in results]
        coh_scores = [r["coh_metrix"] for r in results]
        graph_similarities = [r["graph_similarity"] for r in results]

        aggregated[model] = {
            "bleu_avg": np.mean(bleu_scores),
            "rouge_l_avg": np.mean(rouge_l_scores),
            "semantic_similarity_avg": np.mean(semantic_similarities),
            "meteor_avg": np.mean(meteor_scores),
            "perplexity_avg": np.mean(perplexities),
            "coh_metrix_avg": np.mean(coh_scores),
            "graph_similarity_avg": np.mean(graph_similarities),
        }
    return aggregated

# Main function
if __name__ == "__main__":
    dataset_file = "combined_dataset.json"  # Path to dataset
    dataset = load_dataset(dataset_file)

    metrics = evaluate_dataset(dataset)
    aggregated_results = aggregate_metrics(metrics)

    # Print results
    for model, scores in aggregated_results.items():
        print(f"\nResults for {model}:")
        for metric, value in scores.items():
            print(f"  {metric.upper()}: {value:.4f}")



Results for qwen2vl:
  BLEU_AVG: 0.0396
  ROUGE_L_AVG: 0.1832
  SEMANTIC_SIMILARITY_AVG: 0.5987
  METEOR_AVG: 0.2331
  PERPLEXITY_AVG: 117.0600
  COH_METRIX_AVG: 0.6404
  GRAPH_SIMILARITY_AVG: 0.0129

Results for llava:
  BLEU_AVG: 0.0303
  ROUGE_L_AVG: 0.1445
  SEMANTIC_SIMILARITY_AVG: 0.6142
  METEOR_AVG: 0.2074
  PERPLEXITY_AVG: 167.9500
  COH_METRIX_AVG: 0.5434
  GRAPH_SIMILARITY_AVG: 0.0025

Results for kosmos:
  BLEU_AVG: 0.0550
  ROUGE_L_AVG: 0.2395
  SEMANTIC_SIMILARITY_AVG: 0.5985
  METEOR_AVG: 0.2220
  PERPLEXITY_AVG: 47.7800
  COH_METRIX_AVG: 0.7411
  GRAPH_SIMILARITY_AVG: 0.0000
